# Analyzing networks model

In [35]:
### import libraries

import pandas as pd
import igraph as ig
import numpy as np

In [ ]:
### Read the graph

filename = "name_of_file.gml"
bip_net = ig.read(filename)

# Average degree
avg_degree = bip_net.
print("Avg. degree: ", avg_degree)

In [65]:
# Crear un grafo de ejemplo
graph = ig.Graph.Erdos_Renyi(n=10, m=15, directed=False)
graph.es["weights"] = np.random.random_sample(size=15)

# Imprimir información básica del grafo
print("Información del Grafo:")
print(graph.summary())

# Obtener y mostrar métricas del grafo
print("\nMétricas del Grafo:")
print(f"Número de nodos: {graph.vcount()}")
print(f"Número de aristas: {graph.ecount()}")
print(f"Grado promedio: {ig.mean(graph.degree())}")
print("Peso de Grado promedio: ", ig.mean(graph.es["weights"]))
print("Degree distribution")
print(f"Densidad: {graph.density()}")
print(f"Diámetro del grafo: {graph.diameter()}")
print(f"Average path lenght: {graph.average_path_length()}")
print(f"Componentes conectados: {len(graph.components().subgraphs())}")
print(f"Coeficiente de clustering: {graph.transitivity_undirected()}")
print(f"Coeficiente de asortatividad: {graph.assortativity_degree()}")

# Degree Correlation

#print(f"Centralidad de intermediación (betweenness centrality): {graph.betweenness()}")
#print(f"Excentricidad de cada nodo: {graph.eccentricity()}")
#print(f"PageRank: {graph.pagerank()}")
#print(f"Coeficiente de eigenvector de cada nodo: {graph.eigenvector_centrality()}")

Información del Grafo:
IGRAPH U--- 10 15 -- 
+ attr: weights (e)

Métricas del Grafo:
Número de nodos: 10
Número de aristas: 15
Grado promedio: 3.0
Peso de Grado promedio:  0.5296715524249522
Degree distribution
Densidad: 0.3333333333333333
Diámetro del grafo: 3
Average path lenght: 1.8222222222222222
Componentes conectados: 1
Coeficiente de clustering: 0.20930232558139536
Coeficiente de asortatividad: -0.6333938294010888


In [64]:
import igraph as ig
import matplotlib.pyplot as plt
import numpy as np

# Crear un grafo de ejemplo (puedes reemplazarlo con tu propio grafo)
graph = ig.Graph.Erdos_Renyi(n=100, m=150, directed=False)

# Obtener la distribución de grados del grafo original
degree_distribution = graph.degree_distribution()

# Calcular la correlación de grados entre el grafo original y su grafo aleatorio
random_graph = ig.Graph.Degree_Sequence(degree_distribution)
degree_correlation_original = graph.degree_correlation()
degree_correlation_random = random_graph.degree_correlation()

# Graficar la correlación de grados
plt.plot(degree_correlation_original, label="Grafo Original", color="blue", marker="o")
plt.plot(degree_correlation_random, label="Grafo Aleatorio", color="red", linestyle="--", marker="s")

# Configuraciones adicionales
plt.title("Correlación de Grados")
plt.xlabel("Grado k")
plt.ylabel("Correlación de Grados")
plt.legend()
plt.grid(True)
plt.show()


TypeError: sequence or iterable expected

In [ ]:
ig.Graph.degres